# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [25]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [26]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [27]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)
    # 用Selenium模擬下拉網頁動作，讓網頁更新
import time
time.sleep(5)
SCROLL_PAUSE_TIME = 2
    # 取得目前網頁height(捲動網頁height會增加)
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    # 滾到底部
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    print('網頁更新中...')
    # height不再增加 表示已經到底
    if new_height == last_height:
        print('到達頁面底端')
        break
    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [28]:
# 2.取得資料（等瀏覽器資料出現後才執行）：丟到 BeautifulSoup 解析
    # 自訂函數 抓取外部連結 新聞內容
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class': 'story'}).find_all("p")
    for p in news_content:
        if (p.string is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [33]:
    # 抓取網頁內容
import time
time.sleep(10)
html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")
# 3.關閉瀏覽器
browser.quit()

# 4.抓每篇新聞
all_news = soup.find("div", attrs={'class': 'block block_1 infinite_scroll'})
news_block = all_news.find_all('div', attrs={'class': 'piece clearfix'})
for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    # 連到外部連結，擷取詳細新聞內容
    try:
        externalLink = "https://www.ettoday.net" + news_body.a["href"]
        getNewsDetailContent(externalLink)
    except:
        continue

----------------------------------------------------------------------

[0] 借2億擔保金卻沒錢還　許金龍再遭羈押重回看守所

▲許金龍還不出保證金2億元，再遭羈押。（圖／記者吳銘峯攝）
記者吳銘峯／台北報導
前股票上櫃公司樂陞科技董事長許金龍，爆發與日商「百尺竿頭」公司違約不交割案件，二審將他判刑12年，而許金龍也籌足2億保金獲釋。但他這2億保金其實是向姊夫所借，因為無力償還，姐夫要求法院退還保金。法院審理後，24日下午裁定退還保金，但重新羈押許金龍。
樂陞科技一度榮登遊戲類股股后，2016年中傳出將被日商「百尺竿頭」公司以48億餘元公開收購，成為台灣史上遊戲業最大收購案。但最後「百尺竿頭」公司卻違約不交割，未履約付款，成為我國證券交易史上首宗公開收購違約交割案件。案發後檢調單位隨即將樂陞董事長許金龍傳喚到案，並聲請羈押獲准。
檢方最後將許金龍提起公訴，一審法院將將許金龍判刑18年、併科1億元罰金。高法院二審去年11月底宣判，仍將他重判有期徒刑12年、併科罰金1億元，並宣告沒收12.1億。高院隨即裁准許金龍2億元交保，並限制住居、出境、出海，且應於每日晚間8點至9點間至轄區派出所報到。但許金龍一直籌不出保金，直到今年3月間，才順利籌出保金，離開看守所。
但其實許金龍的2億保金乃姐夫畢華民所出具，許金龍交保後，一直無力償還保金。畢華民因此向高等法院聲請退還保金，高院24日上午開庭審理，並傳喚許金龍的哥哥許世龍到庭。由於當時保金乃許世龍與畢華民簽訂借貸契約書，由畢華民出資2億元，並擔任具保人，另外契約書上也載明，若交保後6個月內未還款，許金龍願意再被執行羈押，許金龍出監後也簽名同意擔任連帶保證人。
----------------------------------------------------------------------

[1] 智慧機械連二年產值破兆　經長：續推智慧製造產業升級

▲經濟部長沈榮津表示，未來將透過投資台灣三大方案與產創條例投資抵減，持續推動以智慧機械驅動智慧製造，以帶動產業升級。（圖／記者屠惠剛攝）
記者林淑慧／台北報導
經濟部長沈榮津今（24）日表示，在推動中小企業數位化、標竿企業智慧化政策下，智慧機械產業連續二年產值已突破兆元，未來將透過投資台灣三大方案與

▲敏盛醫捐贈台大護理系校舍。（圖／敏盛提供）
記者姚惠茹／台北報導
盛弘醫藥（8403）旗下敏盛醫療體系今（24）日與國立台灣大學舉辦簽約儀式，由敏盛醫療體系總裁楊敏盛、執行長楊弘仁與台灣大學校長管中閔、副校長張上淳共同簽署，並以實物捐贈的方式在台大醫學院校地興建護理系校舍，預計五年內完工，將以「敏盛護理健康大樓」命名啟用。
楊敏盛及三個兒女皆是台大畢業生，因此不僅在2015年7月與台大前校長楊泮池簽約捐贈新台幣1.5億元，包括修建醫學校區的護理系館以及整修校總區的第一學生活動中心「怡仁堂」，而且已在2016年底完工啟用。
敏盛總裁楊敏盛表示，醫師及護理師是攸關全民健康福址的專業人士，需要投注更多的支持與資源，尤其近年來醫病關係不佳，醫護人員的價值被貶抑，護理人員大量流失，造成護士荒，也洐生醫院減床以致影響醫療品質等問題。
敏盛表示，新建大樓坐落於台大城中校區，基地面積約2萬3千平方公尺，基地南臨徐州路、東接紹興南路、北側為女四舍、西側為公共衛生大樓，而興建範圍內允建建築面積為424坪、允建容積樓地板面積為2,545坪，並依需求興建地上七至八層教學研究空間，地下層則為地下二至三層停車空間。
敏盛指出，其實在4年前即已捐贈1.5億元給台灣大學，除了回饋母校台灣大學、台大醫學院和台大醫院外，這次再選擇捐贈護理系校舍，更是對護理人員專業的尊重與支持，也希冀吸引更多優秀青年投入醫療護理的領域。
----------------------------------------------------------------------

[11] 東森購物公佈10大熱銷商品排行榜　 安瓶熱賣可疊106座101大樓

▲東森購物發佈年度10大熱銷商品排行榜。（圖／東森購物提供，以下同）
記者洪菱鞠／台北報導
東森購物公布2019年「10大熱銷商品排行榜」，賣出最多件數冠軍是「KOPIKO集團高機能咖啡組」，共售出68,060組、2,720,000包咖啡；從營業額來看，「東森自然美水漾超彈潤口服安瓶14天神效組」則是熱銷第1，共賣出675,000瓶、疊起來有106座101高，創造1.89億元驚人成績，前10名當中，美容保健類佔了7項，可見消費者對美及健康的重視，另3項是「英國KANGOL 80周年限定後背包」、「王品集團 Tasty西堤牛排餐」券和「王品集團 陶板屋餐券」，累積銷

▲長榮航空為優化機隊持續引進波音787夢幻客機，今年再度獲得AirlineRatings.com評選為全球最佳航空公司第八名之肯定。（圖／長榮航空提供）
記者林淑慧／台北報導
航空業搶客出新招！長榮航空今(24)日表示，隨著787夢幻客機陸續交機，機隊的運用將更具彈性，為提升軟體服務力，正式導入AI文字客服機器人，從官網或APP供民眾查詢近500項專業知識，以AI智能提供零時差的數位服務，以期提升旅客黏著度。
北美是長榮航空的主力市場之一，現有紐約、洛杉磯、芝加哥、舊金山、休士頓、西雅圖、溫哥華及多倫多共八條航線往返北美，每週可提供超過82個航班的服務，除了是台灣民眾赴美的首選，現在國外旅客搭乘的比例也逐年提升，挹注公司營運表現。
不過，北美市場的長程航線具有時差，長榮航空表示，導入文字客服機器人系統，將可因應新世代數位通路服務管道的需求，也能提高客服同仁的服務水準與效率。
▲長榮航空進入AI智能時代，運用人工智慧，提供24小時文字客服機器人的服務，提供旅客各類資訊。（圖／長榮航空提供）
長榮航空指出，文字客服機器人與公司後台系統充分整合，提供最即時與正確的資料，由AI智能客服為第一線面對旅客，提供訂位票務、班機到離時刻、特殊需求、官網及App購票諮詢、行李額度諮詢、基本機場運等近500項專業知識查詢，讓旅客可以用更直覺、彈性與自主的提問方式查詢各類服務。
如果AI智能客服無法回答問題時，旅客只要以會員身分完成登入，就可於正常上班時間轉接為專人服務，如此由AI智能客服與真人服務協同合作，將有效提升客服的效率。
展望後市，法人預估在787新世代飛機投入高運量及中長程航線下，長榮航空可望增加產品競爭力，另導入新一代票價系統，以差異化票價產品滿足不同客群，以提升品質來增加航班收益，力拚全年獲利可期。
依據財報，長榮航空今年第3季稅後純益為12.71億元，雖然仍較去年同期腰斬，但比起第2季獲利只有3,319萬元，季增率高達37.5倍；單季每股獲利0.26元，與第二季只有0.01元相比，獲利已大幅改善。
其他人也看了這些～
►ETtoday 8歲歡慶！優惠好康都在這裡～～
----------------------------------------------------------------------

[22] 惠友新捷境　6大機能一次到位　慢工細活打造只為成就最

▲兆勁董事長紀政孝。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
網通廠兆勁（2444）董事長紀政孝今（23）日表示，看好明年5G元年的半導體商機爆發，預計將維持固有網通產品的基礎之下，逐步轉型朝向記憶體、半導體再生晶圓及IC晶片設計生產公司等面向拓展，最大目標就是力拚轉虧為盈。
紀政孝表示，兆勁整體營運目前仍以網通事業為主，但是預計明年網通和半導體的比重將會從今年的7:3變為4:6，增加記憶體模組、光速光通訊雷射晶片(VCSEL Chip)、減薄再生晶圓的比重，並將以品牌「Abocom」打出知名度，目前已經有出貨給中國大陸、印度和中東市場。
紀政孝指出，兆勁營運受惠新產品的記憶體模組、減薄再生晶圓材料、VCSEL Chip、Mini LED等業績接單持續暢旺挹注下，從今年7月以來，已有4個月繳出單月營收歷史新高的亮麗成績，明年業務將逐步轉型朝向記憶體、半導體再生晶圓及IC晶片設計生產公司等面向拓展。
紀政孝說明，其中記憶體模組事業(Memory Testing & Production)更是目前經營團隊的強項業務，將以自有品牌AboCom銷售Micro-SD記憶卡、SSD固態硬碟、USB隨身碟、DRAM等產品，並從今年10月開始已增加相關業務，目前也通過Wal-Mart供應商驗廠認證，可望帶動出貨量逐月成長。
紀政孝補充，減薄再生晶圓材料(Advanced Wafer)業務方面，已獲得日本半導體公司認證完成，而VCSEL Chip則在中美貿易戰中國品牌商去美化大趨勢下，已有多家國內外廠商陸續認證中，目前業務接單暢旺有機會挹注營運加溫。
展望明年營運，紀政孝表示，兆勁受惠新產品記憶體模組、減薄再生晶圓材料、VCSEL Chip業績接單，以及在5G新產品高速先進雷射晶片與半導體相關產業中，兆勁今年已投入大量資本支出，明年將可減少相關部分費用支出，營運可望力拚轉虧為盈。
----------------------------------------------------------------------

[36] 星展銀估G3國家2020年暫停降息　股債匯市投資重點一次看

▲星展銀預估，G3國家央行明年將暫停降息，但仍維持貨幣寬鬆低利環境，股市可望受惠。（示意圖／CFP）
記者紀佳妘／台北報導
星展銀行（DBS）今（23）日發布2020年第1季投資展望，在公債殖

▲Google新任執行長皮查伊（Sundar Pichai）。（圖／達志影像）
記者謝仁傑／綜合報導
現任Google執行長皮查伊（Sundar Pichai）成為全球薪資最高的高階管理人之一。皮查伊在過去的幾年中擔任Google的CEO，年薪為65萬美元（約新台幣1964萬元）；然而根據美國證券交易委員會（SEC）提交的文件顯示，2020年皮查伊的年薪將增至200萬美元（約新台幣6042萬元），比原來年薪增加了2倍左右。
另外，他還有機會領取高額股票分紅，如果達標，未來幾年最高可領走價值2.4億美元（約新台幣72.5億元）的分紅。
皮查伊在兼任Google和母公司Alphabet的CEO後，職責擴大，像其他高階主管一樣，皮查伊在滿足公司特定績效指標或績效股單位（PSU）後，還可領股票報酬。
另外，如果Alphabet股價表現超越同期的S&P 100指數，每年約可多拿價值4500萬美元（約新台幣13.6億元）的股票，兩年合計9千萬美元。
換句話說，未來三年，Pichai如果業績達標，獲得的分紅總值將達2.4億美元（約新台幣72.5億元）。
Google聯合創辦人佩奇（Larry Page）與布林（Sergey Brin）各持有Alphabet約5.8%、5.6%股份，新的接班人皮查伊持股約0.1%，這意味著新任執行長仍會受到董事會包括創始人的掌控。加上近期美國各州對於反壟斷及隱私權的調查，以及歐洲最近對於Google獨佔市場的判決，皮查伊這個執行長雖然領得多，也確實不好當。
----------------------------------------------------------------------

[44] 主計總處：11月失業率3.73%　就業人數增幅創10年新低　

▲主計總處國勢普查處副處長潘寧馨。（圖／記者余弦妙攝）
記者余弦妙／台北報導
行政院主計總處今（23）日發布最新人力資源調查結果，11月失業率為3.73%，較上月下降0.04百分點，較上年同月則升0.03個百分點。1至11月失業率平均為3.74%，較上年同期上升0.03個百分點。另外，主計總處國勢普查處副處長潘寧馨表示，這次就業人數增幅為創下民國10年來同期最低，主要就是因為少子化的關係。
11月勞動力人數為1,197萬3千人，較上月增加 5 千人或 0.04%，較去年同月增5萬7

▲美國科技大廠IBM。（圖／路透）
財經中心／台北報導
IBM宣稱已研發出一種新電池技術，3種原料可從海水萃取，是前所未有的組合，可以取代鎳跟鈷等重金屬，比現有的鋰電池更安全，更便宜，還可以在短短5分鐘內充電至80％的電池狀態。
IBM已和戴姆勒旗下的賓士研究部門、日本電池電解質供應商中央玻璃株式會社、以及電池製造商Sidus等公司合作，共同研發這款下一代電池設計，IBM說，儘管目前還處於探索階段，不過希望能夠實現這個研發計畫。
目前的大多數電池，都由重金屬製成，近年來，引發了不少污染環境的質疑，而且還有低薪聘僱非洲童工在鈷礦場工作的人道爭議，也因此，如果能有更先進的電池出現，勢必成為一大創新革命，而且有助於電動車的研發升級。
▲ 美國科技大廠IBM宣稱已研發出一種新電池技術，圖為IBM研究人員在IBM研究電池實驗室工作，結合和測試獨特的材料和配方（圖／翻攝自IBM官網）
其他人也看了這些～
►ETtoday 8歲歡慶！優惠好康都在這裡～～
----------------------------------------------------------------------

[55] 財政部射3箭迎資金回台　北富銀調查：逾75%客戶有意將資金匯回

▲北富銀自11月29日起於台北、台中、高雄一連舉辦4場「資金返鄉路 富邦領航行」巡迴講座，解析海外資金回台的專法及解釋令的規定、常見的疑難雜症。（圖／北富銀提供）
記者紀佳妘／台北報導
富邦金（2881）旗下台北富邦銀行最新調查發現，超過75%客戶有意將海外資金匯回台灣，且逾80%關切個人身分申請資金回台議題。北富銀對此指出，政府祭出專法優惠稅率鼓勵資金回台，磁力效應已發酵，近期諮詢需求明顯增多，這次與客戶面對面，疏通資金回台法令及實務卡關痛點，並針對回台資金提供個人化的規劃諮商。
全球反避稅政策浪潮，推升海外資金的稅務風險，不少人想藉助財政部「三支箭」，包括歡迎台商回台投資行動方案、發布解釋令、實施境外資金匯回管理運用及課稅條例，將資金匯回台灣。不過，北富銀表示，多數客戶反映，依循專法將資金匯回台灣，儘管流程簡易，但為了對相關稅務法令有正確的理解與整體性的評估，因此近日舉辦「資金返鄉路 富邦領航行」巡迴講座，邀請持有海外資產的客戶參加。
北富銀透過問卷調查發現，超過75%與會者有意將海外資金匯回台灣，且逾80%

▲近半的亞太區企業，一年之內有併購計畫，顯見貿易戰未打亂亞太企業的併購計畫。（示意圖／Pixabay）
記者吳靜君／台北報導
安永聯合會計師事務所今（23）日公布「全球資本信心晴雨表」，調查結果顯示，52%的全球企業未來一年內會進行收購，以調整他們的投資組合；全球仍有併購企圖，主要是因為對數位科技等新科技與人才的需求；另外亞太地區有49%企業未來一年內有收購計畫，超過過去十年的平均值，顯見併購意願仍高。
安永的報告調查對象涵蓋45國、共2,900家全球企業高階主管，每兩年調查一次。以亞太地區來說，有49%亞太地區的高階主管表示，未來一年之內有收購計畫，此比率超過自2009年的亞太地區10年調查歷史平均值。
由併購標的地區來看，受訪的亞太企業最有意願併購美國、中國大陸、英國地區標的；全球企業看好前十大併購標的中，則有四個國家位於亞太地區，為中國大陸在第4位、澳洲第8位、 日本第9位、新加坡第10位。
至於台灣方面，安永財務管理諮詢服務公司總經理何淑芬表示，台灣已從高成長開發國家進入成熟已開發經濟體，經濟成長率相較開發中的國家低，加上台灣融資成本較低廉，因此台灣企業向外尋求成長動能，或透過向外併購以取得新科技，也是近期看到台灣企業向外走的主要原因之一。
亞太區熱門投資領域　媒體娛樂業首次擠入前三大
67%受訪亞太企業已投資數位及科技資產以保持成長動力，除了收購外，也持續透過企業創投資金以及外部資金來投資新科技。根據此次調查，亞太區企業未來一年內最熱門的前三大投資領域，分別為生物科技、科技，以及第一次上榜的媒體娛樂業。
何淑芬指出，新科技及新創出現，重新定義了企業所處的競爭環境。企業必須要建造、合作、投資在科技以鞏固未來成長。目前大環境情勢雖會影響到企業成長，但也是讓台灣企業重新審視策略及規劃，以確保可以在這波經濟情勢中，抓住未來長期的成長契機。
----------------------------------------------------------------------

[67] 李牮斯／國際CRS稅務架構的實施？台灣成為「新瑞士」的契機

▲財政部發佈的「共同申報及盡職審查作業辦法」、「租稅協定稅務用途資訊交換作業辦法」等等，政府近年在跟進金融合規現代化的實質努力，亦受到APG等國際組織的認可。（圖／記者陳韋帆攝）
●李牮斯／避險基金經理人，新世界智庫

▲大魯閣實業總經理謝國棟（圖中）表示，未來集團主力為差異化的運動休閒主題體驗型商場，力拚明年營運能損益兩平或轉虧為盈。（圖／記者林淑慧攝）
記者林淑慧／台北報導
大魯閣實業總經理謝國棟表示，目前營運主軸為商場開發經營、休閒娛樂事業等，旗下鈴鹿賽道樂園今年入園已達400萬人次，遊客二次回訪率達五成，未來集團主力為差異化的運動休閒主題體驗型商場，力拚明年營運能損益兩平或轉虧為盈。
大魯閣目前營運主軸為商場開發經營、休閒娛樂事業、休閒度假村、中國大陸事業，近年以「輕資產、重經營」策略，以主題式體驗型購物中心為主力業務，並進軍大陸購物中心，分別在上海、杭州、蘇州、成都、重慶都有直營點，合作夥伴都選擇大陸前30大的地產開發。
謝國棟指出，台日合作的高雄鈴鹿賽道樂園，取得日本鈴鹿賽車場技術與授權，引進日本10:1等比縮小的擬真卡丁賽道，吸引卡丁車友爭相朝聖，更結合親子共駕，為樂園最知名的遊樂設施之一。
不過，從財務面來看，鈴鹿賽道樂園的獲利模式尚未建立，自開業以來持續虧損，謝國棟表示，公司已計劃將鈴鹿賽道樂園進行大幅度的資產減損，減虧方式內部仍在研議中，而大陸事業也持續調整體質，希望找到在地合作夥伴以提升商場獲利能力，力拚讓集團營運損益兩平。
展望後市，大魯閣表示，未來仍將聚焦於商場事業，以差異化的運動休閒主題體驗型商場開發及經營為方向，攜手富邦集團、健身工廠、國賓影城等合作夥伴，以體驗性業態與實體通路結合的方式，搶攻運動體驗商機。
依據財報，大魯閣實業2019年前三季合併營收達9.09億元，較去年同期的8.11億元有所成長，毛利率達84.48%亦優於去年同期的68.75%；惟近年持續虧損，今年前三季虧損達1.91億元，每股稅後淨損達1元。
其他人也看了這些～
►ETtoday 8歲歡慶！優惠好康都在這裡～～
----------------------------------------------------------------------

[77] 歐元匯價跌破34元！創超過2年半新低　換新台幣5萬多賺3千元

▲ 歐元匯價來到33元字頭。（示意圖／pixabay）
記者李瑞瑾／台北報導
歐元匯價跌破34元整數關卡！根據台銀今（23）天上午牌告的現金賣出匯價，歐元來到33.94元，創下2年7個月以來新低。相較今年6月高點匯價，如今可以多換到90歐元，等於多賺到新台幣

▲86%企業年終人人有獎，若提前遞辭呈獎金損失估計45K。（圖／取自免費圖庫Pixabay）
記者余弦妙／台北報導
年底一到員工最在意的就是「年終獎金」，根據根據yes123求職網做的「企業年終獎金與勞工尾牙期望調查」顯示，對於企業年終發放方式，有高達86%，屬於「所有正職員工人人有獎」；同時在獎金基數部分，有68.4%的公司透露，有高低的差異；剩下31.6%，屬於沒有高低的差異，也就是所謂的齊頭式獎金。
根據yes123求職網表示，不論發或不發，年終獎金的基數平均值，今年落在1.31個月，相較於去年的1.35個月，下降0.04個月，等於少了1.2天，仍是六年以來的第三高；同時扣除「不發放的」(比例佔9.6%)，整體平均值則變成1.45個月。
若以行業別進行交叉分析，年終獎金較高的則有，「金融保險與會計統計業」(2.4個月)、「科技資訊業」(1.7個月)、「運輸與物流倉儲業」(1.56個月)，以及「傳統製造業」(1.53個月)、「醫護與生技業」(1.46個月)。
反之，年終獎金較低的則有，「大眾傳播與公關廣告業」(0.57個月)、「文教業」(0.7個月)、「農林漁牧礦與環保生態業」(0.86個月)，以及「民生服務業(不含餐宿休旅)」(0.9個月)、「餐飲住宿與休閒旅遊業」(1個月)。
此外，有30.9%的上班族評估，若在鼠年過年前提出辭呈，對年終獎金或分紅「不會有任何變化」；只是認為會有損失的，合計也佔了67.2%，其中38%認為會完全拿不到；29.2%則認為會減少；極少數1.9%，認為會增加，同時平均預估，會因此損失掉4萬5千元的獎金或分紅。
對於勞方的擔憂，資方看法又是如何？根據yes123求職網調查顯示，有高達55.4%的企業指出，如果在發年終獎金前，得知某員工想離職，給他(她)的金額，「不會重新評估，完全照工作表現而定」；其餘有34.4%表示，「會重新評估，少給一些或完全不給」；僅有10.2%透露，「會重新評估，想留住人才，所以多給一些」。
----------------------------------------------------------------------

[89] 運動行銷台灣精品　貿協前進花蓮「聖誕健走」吸引1400人

▲(左起)花蓮縣商業會理事長張智欽、國貿局主任秘書倪克浩、外貿協會行銷專案處處長吳俊澤與民眾合影。（圖／貿協